In [65]:
import os
import sys
import numpy as np
from utils import read_raster
from rasterize import get_lidar_tiles
from colin_inv import *

In [66]:
# loading raster
rast = np.memmap('BK_raster.bin', dtype = 'float', mode = 'r')

In [67]:
rast_bk = rast.reshape(56162, 36697)

In [157]:
params = np.array([4.58626623e+00,  -5.60405800e-02,   1.03110986e-13,
                    9.87909846e+05,   1.91300342e+05,   2.37734868e+02,
                    1.46392513e+04])
x0     = params[3]
y0     = params[4]

In [158]:
nrow = 4096
ncol = 3072

# initialize grids
img = np.zeros((nrow, ncol), dtype=float)
xgrid = np.zeros((nrow, ncol), dtype=float)
ygrid = np.zeros((nrow, ncol), dtype=float)

In [159]:
# [[coords.xmin.min(), coords.ymin.min()],[coords.xmax.max(), coords.ymax.max()]]
mm = [[971532.16571600002, 152240.50694399999], [1008227.35159, 208401.43444800001]]

In [160]:
rs = np.arange(0, 50000., 10.).reshape(5000, 1)

In [161]:
#Trying without centering
for ii in range(nrow)[::10]:
    print("\r{0} : {1}".format(ii+1, nrow)),
    sys.stdout.flush()
    for jj in range(ncol)[::10]:
        xx, yy, zz = colin_inv_rad(params, ii, jj, rs)

4091 : 4096


In [162]:
xx.shape

(5000, 1)

In [166]:
rind  = (yy - mm[0][1]).round().astype(int)
cind  = (xx - mm[0][0]).round().astype(int)

In [167]:
# this is to limit x, y at distances that are covered by the raster?
tind  = (rind >= 0) & (cind >= 0) & (rind < rast_bk.shape[0]) & (cind < rast_bk.shape[1])

In [168]:
np.unique(tind, return_counts=True)

(array([False,  True], dtype=bool), array([ 751, 4249]))

In [169]:
rind.shape

(5000, 1)

In [170]:
cind.shape

(5000, 1)

In [172]:
rind  = rind[tind]
cind  = cind[tind]
xx    = xx[tind]
yy    = yy[tind]
zz    = zz[tind]

In [173]:
zz

array([  237.734868  ,   239.21468427,   240.69450054, ...,  6521.03475988,
        6522.51457615,  6523.99439243])

In [174]:
xx

array([ 987909.846     ,  987905.99095243,  987902.13590486, ...,
        971541.31401788,  971537.45897031,  971533.60392274])

In [175]:
rind

array([39060, 39051, 39042, ...,   389,   380,   371])

In [176]:
cind

array([16378, 16374, 16370, ...,     9,     5,     1])

In [177]:
#np.unique(rast_bk[rind, cind], return_counts=True)

In [178]:
rast_bk[rind, cind]

array([  0.    ,  93.42  ,  23.8675, ...,   0.    ,   0.    ,   0.    ])

In [179]:
tall  = rast_bk[rind, cind] > zz

In [180]:
np.unique(tall, return_counts=True)

(array([False], dtype=bool), array([4249]))

In [53]:
tall.max()

True

In [54]:
dd    = (xx[tall] - x0)**2 + (yy[tall] - y0)**2

In [55]:
dd

array([     0.        ,   2448.54631178,   4799.1507711 ,  11850.96414904])

In [56]:
close = np.arange(rind.size)[tall][dd.argmin()]

In [57]:
close

0

In [58]:
img[ii, jj] = dd.min()**0.5

In [59]:
img

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [61]:
xx[close]

987922.03610049537